In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os

plt.ion()   # interactive mode


In [19]:
resnet18 = torchvision.models.resnet18(pretrained=True)

In [22]:
resnet34 = torchvision.models.resnet34(pretrained=True)

In [16]:
image = Variable(torch.randn(10,3,300,300))

In [24]:
resnet34(image)

Variable containing:
-1.7810  0.6528 -2.4912  ...  -1.8502  1.3592 -0.4384
-0.3808 -1.1865  0.0784  ...  -1.1300  0.8547  0.6620
 1.7278  2.1177  1.6187  ...  -1.4556  1.6320  1.8486
          ...             ⋱             ...          
-0.8924 -3.4939  0.6537  ...  -1.5514  3.3450  0.1086
 1.2976 -0.0061  0.8467  ...  -2.0196  0.7640  1.7193
-0.6872 -2.3337  1.2203  ...  -1.4486  1.0751 -0.8062
[torch.FloatTensor of size 10x1000]

In [85]:
class CNNRNNClassifier(nn.Module):
    def __init__(self, resnet):
        super(CNNRNNClassifier, self).__init__()
        self.input_dim = 1000
        self.hidden_layers = 50
        self.rnn_layers = 2
        self.classes = 5
        self.sample_rate = 12
        
        self.conv = resnet
        self.lstm = nn.LSTM(self.input_dim, self.hidden_layers, self.rnn_layers)
         
        self.dense = nn.Linear(
            in_features=self.hidden_layers, out_features=self.classes)

    def forward(self, video):
        print(video.shape[1])
        frames, width, height, channels = video.shape

        time_steps = int(frames/self.sample_rate)
#         conv_output = Variable(torch.zeros((time_steps,self.input_dim)))
        
        conv_output = video[]
#         for frame in xrange(0,frames,self.sample_rate):
#             conv_output[i] = self.conv(video[frame])
        # Since we are using conv2d, we need to add extra outer dimension
#         for i in enumerate(self.convs):
#             x = x.unsqueeze(1)
#             x = F.relu(conv(x)).squeeze(3)
#             x = x.view(x.size(0), x.size(2), x.size(1))

        out, _ = self.lstm(conv_output)
        last_output = out[-1, :, :]
        return last_output


In [72]:
import torch
import torch.nn as nn
from torch.autograd import Variable

time_steps = 10
batch_size = 1
in_size = 1000
classes_no = 7

model = nn.LSTM(in_size, classes_no, 2)
input_seq = Variable(torch.randn(time_steps, batch_size, in_size))
input_seq2 = Variable(torch.randn(time_steps, batch_size, in_size))

output_seq, _ = model(input_seq)
last_output = output_seq[-1]

loss = nn.CrossEntropyLoss()
target = Variable(torch.LongTensor(batch_size).random_(0, classes_no-1))
err = loss(last_output, target)
err.backward()


In [73]:
model

LSTM(1000, 7, num_layers=2)

In [74]:
last_output

Variable containing:
-0.1305  0.1652  0.2707 -0.0494  0.3337 -0.0544  0.3133
[torch.FloatTensor of size 1x7]

In [75]:
torch.LongTensor(batch_size).random_(0, classes_no-1)


 0
[torch.LongTensor of size 1]

In [87]:
resnet = torchvision.models.resnet18(pretrained=True)
mo = CNNRNNClassifier(resnet)

In [83]:
video = torch.randn((123, 240, 320, 3))

In [84]:
mo(video)

240


ValueError: Expected 4D tensor as input, got 3D tensor instead.

In [104]:
vid = Variable(torch.randn(132,240,320,3))

In [106]:
resnet(vid.permute(0,3,1,2))

Variable containing:
-1.0677e+00  1.4261e-01 -1.4306e+00  ...   5.9041e-01  6.0677e+00  6.1782e-01
-4.2128e-01 -4.3623e-01 -1.9895e+00  ...  -4.7480e-01  2.3236e+00 -6.1480e-01
-3.3293e+00 -1.1774e+00 -1.0442e+00  ...   8.0240e-01  6.4464e-01  8.7894e-01
                ...                   ⋱                   ...                
-1.6050e+00 -1.2811e+00 -2.4277e+00  ...   5.2192e-01  2.5656e+00  1.3144e+00
-2.3960e+00 -7.7419e-01 -4.7761e-01  ...  -1.6319e+00  3.0802e+00  4.4542e+00
 3.1441e+00  3.8713e+00  6.3788e-01  ...   1.0875e+00  2.8195e+00  3.3100e-01
[torch.FloatTensor of size 132x1000]